In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example: 'spark-3.2.2'
spark_version = 'spark-3.2.2'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [816 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelea

In [2]:
!pip install pylint

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 488 kB 5.0 MB/s 
     |████████████████████████████████| 251 kB 21.0 MB/s 
     |████████████████████████████████| 103 kB 62.3 MB/s 
     |████████████████████████████████| 57 kB 5.1 MB/s 
     |████████████████████████████████| 843 kB 49.8 MB/s 


In [3]:
# Create and navigate to the tests directory.
from pathlib import Path
if Path.cwd().name != 'tests':
    %mkdir tests
    %cd tests
# Show the current working directory.  
%pwd

/content/tests


'/content/tests'

In [4]:
# Create a  __init__.py file that will contain that will be used to run our functions. 
# This file will be stored in our pwd (/content/tests)
%%file __init__.py
pass

Writing __init__.py


In [7]:
# Create a heart_health.py file that will contain our functions.
# This file will be stored in our pwd (/content/tests).
%%file heart_health.py
"""
Heart Health Module
"""

from pyspark import SparkFiles # pylint: disable=import-error
from pyspark.sql import SparkSession # pylint: disable=import-error

spark = SparkSession.builder.appName("sparkHeartData").getOrCreate()

def import_data():
    """
    Imports the heart health dataset from S3.
    """

    url = "https://2u-data-curriculum-team.s3.amazonaws.com/nflx-data-science-adv/week-6/heart_health.csv"
    spark.sparkContext.addFile(url)
    source_df = spark.read.csv(SparkFiles.get("heart_health.csv"), sep=",", header=True)
    source_df.createOrReplaceTempView('HEART_HEALTH')

    return source_df

def get_states():
    """
    Transform the Heart Health dataset.
    """

    transformed_df = spark.sql("""
    SELECT DISTINCT
        STATE
    FROM HEART_HEALTH
    """)

    return transformed_df


def get_states_with_above_avg_death_rate():
    """
    Transforms the Heart Health dataset.
    """

    transformed_df = spark.sql("""
    SELECT
        STATE,
        AVG(Death_Rate)
    FROM HEART_HEALTH
    GROUP BY STATE
    HAVING AVG(Death_Rate) > 400
    """)

    return transformed_df


def test_row_count_source():
    """
    Tests the number rows in the source dataset.
    """

    source_df = import_data()
    assert source_df.count() == 799


def test_column_count_source():
    """
    Tests the number of columns in the source dataset.
    """

    source_df = import_data()
    assert len(source_df.columns) == 9


def test_get_states():
    """
    Tests the transform logic to ensure state is the only column.
    """

    transformed_df = get_states()

    assert transformed_df.schema.names == ['STATE']


def test_row_count_get_states():
    """
    Tests the row count of the transformed dataset.
    """

    transformed_df = get_states()
    assert transformed_df.count() == 5


def test_row_count_avg_death_rate():
    """
    Tests the row count of the average death rate transformed dataset.
    """

    death_rate_df = get_states_with_above_avg_death_rate()
    assert death_rate_df.count() == 5


def test_column_count_avg_death_rate():
    """
    Tests the columns count of the average death rate transformed dataset.
    """

    death_rate_df = get_states_with_above_avg_death_rate()
    assert len(death_rate_df.columns) == 2


def test_get_states_vs_avg_death_rate_count():
    """
    Tests to ensure that the counts between the two transformations match.
    """

    df_get_states = get_states()
    df_avg_death_rate = get_states_with_above_avg_death_rate()
    assert df_get_states.count() == df_avg_death_rate.count()



Overwriting heart_health.py


In [8]:
!python -m pylint heart_health.py


-------------------------------------------------------------------
Your code has been rated at 10.00/10 (previous run: 7.03/10, +2.97)

